In [ ]:
#| default_exp components

# Components

In [ ]:
#| export
from html.parser import HTMLParser
from dataclasses import dataclass, asdict

from fastcore.utils import *
from fastcore.xml import *
from fastcore.meta import use_kwargs, delegates

try: from IPython import display
except ImportError: display=None

In [ ]:
from lxml import html as lx

In [ ]:
#| export
def show(xt,*rest):
    if rest: xt = (xt,)+rest
    return display.HTML(to_xml(xt))

In [ ]:
#| export
named = set('a button form frame iframe img input map meta object param select textarea'.split())
html_attrs = 'id cls title style accesskey contenteditable dir draggable enterkeyhint hidden inert inputmode lang popover spellcheck tabindex translate'.split()
hx_attrs = 'get post put delete patch trigger target swap include select indicator push_url confirm disable replace_url on'
hx_attrs = html_attrs + [f'hx_{o}' for o in hx_attrs.split()]

In [ ]:
#| export
def xt_html(tag: str, *c, id=None, cls=None, title=None, style=None, **kwargs):
    kwargs['id'],kwargs['cls'],kwargs['title'],kwargs['style'] = id,cls,title,style
    tag,c,kw = xt(tag, *c, **kwargs)
    if tag in named and 'id' in kw and 'name' not in kw: kw['name'] = kw['id']
    return XT([tag,c,kw])

In [ ]:
#| export
@use_kwargs(hx_attrs, keep=True)
def xt_hx(tag: str, *c, target_id=None, **kwargs):
    if target_id: kwargs['hx_target'] = '#'+target_id
    return xt_html(tag, *c, **kwargs)

In [ ]:
#| export
_g = globals()
_all_ = ['Html', 'Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
    'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
    'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'Link', 'Nav',
    'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
    'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
    'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details', 'Dialog',
    'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
    'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
    'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(xt_hx, o.lower())

In [ ]:
#| hide
both_attrs = html_attrs+hx_attrs

with open('../fasthtml/components.pyi', 'w') as f:
    attrs_str = ', '.join(f'{t}:str|None=None' for t in both_attrs)
    f.write(f"def xt_hx(tag: str, *c, {attrs_str}, **kwargs): ...\n")
    for o in _all_:
        attrs = (['name'] if o.lower() in named else []) + both_attrs
        attrs_str = ', '.join(f'{t}:str|None=None' for t in attrs)
        f.write(f"def {o}(*c, {attrs_str}, **kwargs): ...\n")

For tags that have a `name` attribute, it will be set to the value of `id` if not provided explicitly:

In [ ]:
Form(Button(target_id='foo', id='btn'),
     hx_post='/', target_id='tgt', id='frm')

['form',
 (['button', (), {'hx-target': '#foo', 'id': 'btn', 'name': 'btn'}],),
 {'hx-post': '/', 'hx-target': '#tgt', 'id': 'frm', 'name': 'frm'}]

In [ ]:
#| export
def set_val(tag, attr, val):
    if attr.get('type', '') in ('checkbox','radio'):
        if val: attr['checked'] = '1'
        else: attr.pop('checked', '')
    else: attr['value'] = val

In [ ]:
#| export
def find_inps(html):
    if not html: return []
    tag,cs,attrs = html
    if tag == 'input': return [html]
    res = []
    for c in cs:
        if isinstance(c, list): res.extend(find_inps(c))
    return res

In [ ]:
#| export
def fill_form(form, obj):
    "Modifies form in-place and returns it"
    inps = find_inps(form)
    inps = {attrs['id']:(tag,attrs) for tag,c,attrs in inps if 'id' in attrs}
    for nm,val in asdict(obj).items():
        if nm in inps:
            tag,attr = inps[nm]
            set_val(tag, attr, val)
    return form

In [ ]:
@dataclass
class TodoItem:
    title:str; id:int; done:bool
                
todo = TodoItem(id=2, title="Profit", done=True)
check = Input(Label("Done"), type="checkbox", id="done")
form = Form(Fieldset(Input(id="title"), check, Input(type="hidden", id="id"), Button("Save")))
fill_form(form, todo)

['form',
 (['fieldset',
   (['input', (), {'id': 'title', 'name': 'title', 'value': 'Profit'}],
    ['input',
     (['label', ('Done',), {}],),
     {'type': 'checkbox', 'id': 'done', 'name': 'done', 'checked': '1'}],
    ['input', (), {'type': 'hidden', 'id': 'id', 'name': 'id', 'value': 2}],
    ['button', ('Save',), {}]),
   {}],),
 {}]

In [ ]:
#|export
def fill_dataclass(src, dest):
    "Modifies dataclass in-place and returns it"
    for nm,val in asdict(src).items(): setattr(dest, nm, val)
    return dest

In [ ]:
nt = TodoItem('', 0, False)
fill_dataclass(todo, nt)
nt

TodoItem(title='Profit', id=2, done=True)

In [ ]:
#| export
class _FindElems(HTMLParser):
    def __init__(self, tag=None, attr=None, **props):
        super().__init__()
        self.tag,self.attr,self.props = tag,attr,props
        self.res = []

    def handle_starttag(self, tag, attrs):
        if self.tag and tag!=self.tag: return
        d = dict(attrs)
        if [k for k,v in self.props.items() if d.get(k,None)==v]:
            self.res.append(d.get(self.attr, None) if self.attr else d)

In [ ]:
#| export
def find_elems(s:XT|str, tag=None, attr=None, **props):
    "Find elements in `s` with `tag` (if supplied) and `props`, returning `attr`"
    o = _FindElems(tag, attr, **props)
    o.feed(to_xml(s))
    return o.res

In [ ]:
find_elems(form, 'input', 'value', id='title')

['Profit']

In [ ]:
elem = lx.fromstring(to_xml(form))
elem.xpath("//input[@id='title']/@value")

['Profit']

# Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()